In [1]:
import configparser
import logging
import os
import time
from datetime import datetime

import transf_producao
from database import registrar_extracao
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# Configura o logger
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("teste.log"), logging.StreamHandler()],
)

logger = logging.getLogger()

# Carregar o arquivo de configuração
config = configparser.ConfigParser()
config.read("config.ini")

# Diretórios de destino e download
transformacao_dir = config["Paths"]["transformacao_dir"]
download_dir = config["Paths"]["download_dir"]

# Gerar o nome do arquivo esperado dinamicamente
today = datetime.today().strftime("%Y-%m-%d")
PRODUCAO_FILENAME = "RelatorioSaudeProducao.csv"


def configurar_driver():
    """Configura e retorna o driver do Chrome."""
    options = Options()
    # options.add_argument('--headless')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option(
        "prefs",
        {
            "profile.default_content_settings.popups": 0,  # Desativa popups de download
            "download.prompt_for_download": False,  # Não pergunta onde salvar
            "download.directory_upgrade": True,  # Atualiza o diretório automaticamente
            "safebrowsing.enabled": True,  # Habilita o Safe Browsing
            "profile.default_content_setting_values.automatic_downloads": 1,  # Permite múltiplos downloads
        },
    )
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=options
    )
    return driver


def seleciona_xpath(driver, xpath):
    """Seleciona o elemento com o xpath"""
    try:
        seleciona = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(("xpath", xpath))
        )
        seleciona.click()
        return True
    except StaleElementReferenceException:
        return False


def selecionar_primeiro_item(driver, butao_xpath):
    """Seleciona a primeira coluna para download."""
    try:
        item_xpath = butao_xpath[:-6] + "ul/li[1]/a/label/input"
        # Encontra o botão e clica
        button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, butao_xpath))
        )
        button.click()
        # Seleciona o item
        item = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(("xpath", item_xpath))
        )
        if item.is_selected():
            return True
        item.click()
        return True
    except StaleElementReferenceException:
        return False


def clica_download(driver):
    """Clica no botão de download."""
    try:
        # Iniciar o download
        download_button = driver.find_element(
            By.XPATH,
            '//*[@id="j_idt44"]/div/div[1]/div/div[2]/div[2]/div[5]/div/div/div[2]/button',
        )
        driver.execute_script("arguments[0].click();", download_button)

        # Esperar o botão CSV estar clicável
        csv_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable(
                (
                    By.XPATH,
                    '//*[@id="j_idt44"]/div/div[1]/div/div[2]/div[2]/div[5]/div/div/div[2]/ul/li[2]/a',
                )
            )
        )
        driver.execute_script("arguments[0].click();", csv_button)
        return True
    except StaleElementReferenceException:
        return False


def espera_download(
    mes,
    nome_arq,
    download_dir=download_dir,
    expected_filename=PRODUCAO_FILENAME,
):
    """Espera o download do arquivo."""
    # Caminho completo para o arquivo esperado
    expected_file_path = os.path.join(download_dir, expected_filename)

    # Tempo máximo para esperar o download, em segundos
    timeout = 180
    start_time = time.time()

    # Esperar até que o arquivo apareça na pasta de downloads
    while True:
        if os.path.exists(expected_file_path):
            # Se o arquivo existir, verificar se o download foi concluído
            if not expected_file_path.endswith(".crdownload"):
                logger.info(f"Download completo: {expected_filename}")
                n_mes = mes.replace("/", "-")
                # Mover o arquivo para diretório de dados
                os.rename(
                    expected_file_path,
                    f"{transformacao_dir}/{nome_arq}_{n_mes}.csv",
                )
            return True
        elif time.time() - start_time > timeout:
            # Se o tempo de espera exceder o timeout, exibir uma mensagem de erro
            print(
                f"Tempo limite excedido para download de {expected_filename}."
            )
            return False
        time.sleep(1)  # Esperar 1 segundo antes de verificar novamente


def fazer_download(
    driver,
    mes,
    nome_arq,
    download_dir=download_dir,
    expected_filename=PRODUCAO_FILENAME,
):
    """Faz o download do relatório em formato CSV e espera até que o download esteja completo."""
    logger.info(f"Fazendo download do relatório para {mes}")

    # Inicia o download
    clica_download(driver)

    # Aguarda o arquivo ser baixado
    if not espera_download(mes, nome_arq, download_dir, expected_filename):
        return False
    return True


def seleciona_competencias(driver):
    """Seleciona a competência (mes/ano)"""

    # Clica no botão de de competência (mes/ano)
    competencia_element = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="competencia"]/div/button')
        )
    )
    competencia_element.click()
    # Aguarda a lista de competências aparecer
    competencias = WebDriverWait(driver, 3).until(
        EC.presence_of_element_located(
            ("xpath", '//*[@id="competencia"]/div/ul')
        )
    )
    return competencias


def cria_driver():
    """Cria o driver do Chrome."""
    driver = configurar_driver()
    logger.info("Iniciando o acesso a página de Producao do SISAB")
    driver.get(
        "https://sisab.saude.gov.br/paginas/acessoRestrito/relatorio/federal/saude/RelSauProducao.xhtml"
    )
    return driver




In [9]:
def seleciona_sigtab(driver, codigo):
    """Seleciona o SIGTAB"""
    try:
       # Clicar no botão de adicionar SIGTAB
        sigtab_button = driver.find_element(
            By.XPATH,
            '//*[@id="btnAddSigtap"]',
        )
        driver.execute_script("arguments[0].click();", sigtab_button)

        # escrever o código do SIGTAB
        sigtab_input = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="td-ls-sigtap_filter"]/label/input')
            )
        )

        # Preencher o campo com o código do SIGTAB
        sigtab_input.send_keys(codigo)

        # seleciona o checkbox
        sigtab_checkbox = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="td-ls-sigtap"]/tbody/tr/td/label/input')
            )
        )

        if not sigtab_checkbox.is_selected():
            sigtab_checkbox.click()
        

        return True
    except StaleElementReferenceException:
        return False


def seleciona_cid(driver, codigo):
    """Seleciona o SIGTAB"""
    try:
       # Clicar no botão de adicionar SIGTAB
        sigtab_button = driver.find_element(
            By.XPATH,
            '//*[@id="btnAddSigtap"]',
        )
        driver.execute_script("arguments[0].click();", sigtab_button)

        # escrever o código do SIGTAB
        sigtab_input = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="td-ls-sigtap_filter"]/label/input')
            )
        )

        # Preencher o campo com o código do SIGTAB
        sigtab_input.send_keys(codigo)

        # seleciona o checkbox
        sigtab_checkbox = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="td-ls-sigtap"]/tbody/tr/td/label/input')
            )
        )

        if not sigtab_checkbox.is_selected():
            sigtab_checkbox.click()
        

        return True
    except StaleElementReferenceException:
        return False
    


def executar_sigtap(linha, coluna, codigo, nome_arq):
    """Executa o download dos relatórios de produção para cada mês."""
    # Inicializa o driver
    driver = cria_driver()
    # Esperar a página carregar
    wait = WebDriverWait(driver, 10)

    # Clica no botão de de competência (mes/ano)
    competencia_element = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="competencia"]/div/button')
        )
    )
    driver.execute_script("arguments[0].click();", competencia_element)

    competencias = wait.until(
        EC.presence_of_element_located(
            ("xpath", '//*[@id="competencia"]/div/ul')
        )
    )
    falhas = 0
    for j in range(len(competencias.find_elements(By.TAG_NAME, "li"))):
        # recarregar competencias
        if j > 0:
            competencias = seleciona_competencias(driver)
        for i, competencia in enumerate(
            competencias.find_elements(By.TAG_NAME, "li")
        ):
            if i == j:
                mes = competencia.text
                # pegar o horario de inicio em timestamp

                logger.info(f"Processando o mês {mes}")
                # Clica no botão de de competência (mes/ano)

                competencia.click()

                # Selecionar "Municípios" no dropdown de Linhas
                seleciona_xpath(driver, linha)

                # Selecionar "Condição Avaliada" no dropdown de Colunas
                seleciona_xpath(driver, coluna)

                # Selecionar o SIGTAB
                seleciona_sigtab(driver, codigo)

                

        break

    time.sleep(10)
    logger.info("Script Finalizado")
    # Fecha a janela do navegador
    driver.quit()

"""
executar_sigtap(
    '//*[@id="selectLinha"]/optgroup[1]/option[5]',
    '//*[@id="selectcoluna"]/optgroup[4]/option[3]',
    '0301040141', 'teste')
"""


2024-08-25 16:41:39,306 - INFO - ====== WebDriver manager ======
2024-08-25 16:41:39,369 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-25 16:41:39,409 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-25 16:41:39,443 - INFO - Driver [/home/daniel/.wdm/drivers/chromedriver/linux64/128.0.6613.84/chromedriver-linux64/chromedriver] found in cache
2024-08-25 16:41:40,027 - INFO - Iniciando o acesso a página de Producao do SISAB
2024-08-25 16:41:42,528 - INFO - Processando o mês JUN/2024


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=128.0.6613.84)
Stacktrace:
#0 0x55dc25eb981a <unknown>
#1 0x55dc25b87c91 <unknown>
#2 0x55dc25bd1618 <unknown>
#3 0x55dc25bcfa8d <unknown>
#4 0x55dc25bfc492 <unknown>
#5 0x55dc25bcbe89 <unknown>
#6 0x55dc25bfc65e <unknown>
#7 0x55dc25c1a9c6 <unknown>
#8 0x55dc25bfc233 <unknown>
#9 0x55dc25bca093 <unknown>
#10 0x55dc25bcb09e <unknown>
#11 0x55dc25e80a7b <unknown>
#12 0x55dc25e84a31 <unknown>
#13 0x55dc25e6c645 <unknown>
#14 0x55dc25e855a2 <unknown>
#15 0x55dc25e5181f <unknown>
#16 0x55dc25ea8618 <unknown>
#17 0x55dc25ea87e2 <unknown>
#18 0x55dc25eb860c <unknown>
#19 0x7fa930cdbac3 <unknown>


In [19]:
def fazer_download(
    driver,
    mes,
    nome_arq,
    download_dir=download_dir,
    expected_filename=PRODUCAO_FILENAME,
):
    """Faz o download do relatório em formato CSV e espera até que o download esteja completo."""
    logger.info(f"Fazendo download do relatório para {mes}")

    # Inicia o download
    clica_download(driver)

    # Aguarda o arquivo ser baixado
    if not espera_download(mes, nome_arq, download_dir, expected_filename):
        return False
    return True

def seleciona_cid(driver):
    """Seleciona o SIGTAB"""
    #try:
       # Clicar no botão de adicionar SIGTAB
    cid_button = driver.find_element(
        By.XPATH,
        '//*[@id="btnAddCid"]',
    )
    
    driver.execute_script("arguments[0].click();", cid_button)

    itens_por_pagina = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="dtBasicExample_length"]/label/select')
        )
    )
    itens_por_pagina.click()

    # Aguardar até que a opção de 100 itens esteja presente e clique nela
    opcao_100_itens = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="dtBasicExample_length"]/label/select/option[@value="100"]')
        )
    )
    opcao_100_itens.click()


    for j in range(20):
        # Selecionar todos os CIDs
        cids = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="dtBasicExample"]/tbody')
            )
        )
        for i in range(len(cids.find_elements(By.TAG_NAME, "td"))):
            cid = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (By.XPATH, f'//*[@id="dtBasicExample"]/tbody/tr[{i+1}]/td[1]/label/input')
                )
            )
            cid.click()
        
        # Clica na próxima página
        proxima_pagina = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="dtBasicExample_next"]/a')
            )
        )
        proxima_pagina.click()
    
    # Clica no botão de concluir
    concluir_button = WebDriverWait(driver, 3).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="modal-default-cid"]/div/div/div[2]/button[2]')
        )
    )
    concluir_button.click()     
    

    return True
    #except StaleElementReferenceException:
    #    return False
    


def executar_cid():
    """Executa o download dos relatórios de produção para cada mês."""
    # Inicializa o driver
    driver = cria_driver()
    # Esperar a página carregar
    wait = WebDriverWait(driver, 10)

    # Clica no botão de de competência (mes/ano)
    competencia_element = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="competencia"]/div/button')
        )
    )
    driver.execute_script("arguments[0].click();", competencia_element)

    competencias = wait.until(
        EC.presence_of_element_located(
            ("xpath", '//*[@id="competencia"]/div/ul')
        )
    )
    falhas = 0
    for j in range(len(competencias.find_elements(By.TAG_NAME, "li"))):
        # recarregar competencias
        if j > 0:
            competencias = seleciona_competencias(driver)
        for i, competencia in enumerate(
            competencias.find_elements(By.TAG_NAME, "li")
        ):
            if i == j:
                mes = competencia.text
                # pegar o horario de inicio em timestamp

                logger.info(f"Processando o mês {mes}")
                # Clica no botão de de competência (mes/ano)

                competencia.click()

                # Selecionar "Municípios" no dropdown de Linhas
                seleciona_xpath(driver, '//*[@id="selectLinha"]/optgroup[1]/option[5]')

                # Selecionar "CID" no dropdown de Colunas
                seleciona_xpath(driver, '//*[@id="selectcoluna"]/optgroup[2]/option[7]')

                # Selecionar o CIDs
                seleciona_cid(driver)

        fazer_download(driver, mes, "producao_cid")                

        break

    time.sleep(10)
    logger.info("Script Finalizado")
    # Fecha a janela do navegador
    driver.quit()


executar_cid()

"""
executar_sigtap(
    '//*[@id="selectLinha"]/optgroup[1]/option[5]',
    '//*[@id="selectcoluna"]/optgroup[4]/option[3]',
    '0301040141', 'teste')
"""


2024-08-27 18:23:31,906 - INFO - ====== WebDriver manager ======
2024-08-27 18:23:31,967 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-27 18:23:32,005 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-27 18:23:32,037 - INFO - Driver [/home/daniel/.wdm/drivers/chromedriver/linux64/128.0.6613.84/chromedriver-linux64/chromedriver] found in cache
2024-08-27 18:23:32,569 - INFO - Iniciando o acesso a página de Producao do SISAB
2024-08-27 18:23:34,936 - INFO - Processando o mês JUN/2024
2024-08-27 18:25:45,358 - INFO - Fazendo download do relatório para JUN/2024


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=128.0.6613.84)
Stacktrace:
#0 0x5621cd9e581a <unknown>
#1 0x5621cd6b3e50 <unknown>
#2 0x5621cd69be20 <unknown>
#3 0x5621cd69bb0d <unknown>
#4 0x5621cd699b11 <unknown>
#5 0x5621cd69a31f <unknown>
#6 0x5621cd6a9e3b <unknown>
#7 0x5621cd6c0204 <unknown>
#8 0x5621cd6c574b <unknown>
#9 0x5621cd69aa2e <unknown>
#10 0x5621cd6c0059 <unknown>
#11 0x5621cd746c2b <unknown>
#12 0x5621cd728233 <unknown>
#13 0x5621cd6f6093 <unknown>
#14 0x5621cd6f709e <unknown>
#15 0x5621cd9aca7b <unknown>
#16 0x5621cd9b0a31 <unknown>
#17 0x5621cd998645 <unknown>
#18 0x5621cd9b15a2 <unknown>
#19 0x5621cd97d81f <unknown>
#20 0x5621cd9d4618 <unknown>
#21 0x5621cd9d47e2 <unknown>
#22 0x5621cd9e460c <unknown>
#23 0x7fdb664daac3 <unknown>


In [21]:
def fazer_download(
    driver,
    mes,
    nome_arq,
    download_dir=download_dir,
    expected_filename=PRODUCAO_FILENAME,
):
    """Faz o download do relatório em formato CSV e espera até que o download esteja completo."""
    logger.info(f"Fazendo download do relatório para {mes}")

    # Inicia o download
    clica_download(driver)

    # Aguarda o arquivo ser baixado
    if not espera_download(mes, nome_arq, download_dir, expected_filename):
        return False
    return True

def seleciona_cid(driver, i):
    """Seleciona o SIGTAB"""
    #try:
       # Clicar no botão de adicionar SIGTAB
    cid_button = driver.find_element(
        By.XPATH,
        '//*[@id="btnAddCid"]',
    )
    
    driver.execute_script("arguments[0].click();", cid_button)

    itens_por_pagina = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="dtBasicExample_length"]/label/select')
        )
    )
    itens_por_pagina.click()

    # Aguardar até que a opção de 100 itens esteja presente e clique nela
    opcao_100_itens = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="dtBasicExample_length"]/label/select/option[@value="100"]')
        )
    )
    opcao_100_itens.click()


    for j in range(150):
        if j >= i*5 and j < (i+1)*5:
            # Selecionar todos os CIDs
            cids = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located(
                    (By.XPATH, '//*[@id="dtBasicExample"]/tbody')
                )
            )
            for i in range(len(cids.find_elements(By.TAG_NAME, "td"))):
                cid = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located(
                        (By.XPATH, f'//*[@id="dtBasicExample"]/tbody/tr[{i+1}]/td[1]/label/input')
                    )
                )
                cid.click()
            
            # Clica na próxima página
            proxima_pagina = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//*[@id="dtBasicExample_next"]/a')
                )
            )
            proxima_pagina.click()
        
        # Clica no botão de concluir
        concluir_button = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="modal-default-cid"]/div/div/div[2]/button[2]')
            )
        )
        if j >= (i+1)*5:
            concluir_button.click()
            break     
    

    return True
    #except StaleElementReferenceException:
    #    return False
    


def executar_cid():
    """Executa o download dos relatórios de produção para cada mês."""
    # Inicializa o driver
    driver = cria_driver()
    # Esperar a página carregar
    wait = WebDriverWait(driver, 10)

    # Clica no botão de de competência (mes/ano)
    competencia_element = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH, '//*[@id="competencia"]/div/button')
        )
    )
    driver.execute_script("arguments[0].click();", competencia_element)

    competencias = wait.until(
        EC.presence_of_element_located(
            ("xpath", '//*[@id="competencia"]/div/ul')
        )
    )
    
    for i in range(30):
        competencias.click()
        competencias = seleciona_competencias(driver)
        
        for i, competencia in enumerate(
            competencias.find_elements(By.TAG_NAME, "li")
        ):
            competencia.click()
            if i == 10:
                break

        # Selecionar "Municípios" no dropdown de Linhas
        seleciona_xpath(driver, '//*[@id="selectLinha"]/optgroup[1]/option[5]')

        # Selecionar "CID" no dropdown de Colunas
        seleciona_xpath(driver, '//*[@id="selectcoluna"]/optgroup[2]/option[7]')

        # Selecionar o CIDs
        seleciona_cid(driver, i)

        fazer_download(driver, '12_meses', "producao_cid")                

        break

    time.sleep(10)
    logger.info("Script Finalizado")
    # Fecha a janela do navegador
    driver.quit()


executar_cid()

"""
executar_sigtap(
    '//*[@id="selectLinha"]/optgroup[1]/option[5]',
    '//*[@id="selectcoluna"]/optgroup[4]/option[3]',
    '0301040141', 'teste')
"""


2024-08-27 19:04:52,901 - INFO - ====== WebDriver manager ======
2024-08-27 19:04:52,956 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-27 19:04:52,991 - INFO - Get LATEST chromedriver version for google-chrome
2024-08-27 19:04:53,025 - INFO - Driver [/home/daniel/.wdm/drivers/chromedriver/linux64/128.0.6613.84/chromedriver-linux64/chromedriver] found in cache
2024-08-27 19:04:53,581 - INFO - Iniciando o acesso a página de Producao do SISAB


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=128.0.6613.84)
Stacktrace:
#0 0x5618c8ef581a <unknown>
#1 0x5618c8bc3c91 <unknown>
#2 0x5618c8c1492f <unknown>
#3 0x5618c8c084d6 <unknown>
#4 0x5618c8c38492 <unknown>
#5 0x5618c8c07e89 <unknown>
#6 0x5618c8c3865e <unknown>
#7 0x5618c8c569c6 <unknown>
#8 0x5618c8c38233 <unknown>
#9 0x5618c8c06093 <unknown>
#10 0x5618c8c0709e <unknown>
#11 0x5618c8ebca7b <unknown>
#12 0x5618c8ec0a31 <unknown>
#13 0x5618c8ea8645 <unknown>
#14 0x5618c8ec15a2 <unknown>
#15 0x5618c8e8d81f <unknown>
#16 0x5618c8ee4618 <unknown>
#17 0x5618c8ee47e2 <unknown>
#18 0x5618c8ef460c <unknown>
#19 0x7f887a30bac3 <unknown>
